In [32]:
import numpy as np
import cv2
# from scipy.ndimage import gaussian_filter
from queue import Queue
from tqdm import tqdm

def efros_leung_synthesis(sample_img, output_size, window_size, threshold=0.1, gaussian_sigma=1.0):
    # Convert sample to grayscale if needed
    if len(sample_img.shape) > 2:
        sample_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2GRAY)
    
    H, W = sample_img.shape
    h_out, w_out = output_size
    
    # Initialize output with a random 3x3 seed from the sample
    output = np.zeros((h_out, w_out), dtype=np.uint8)
    seed_y = np.random.randint(H - 3)
    seed_x = np.random.randint(W - 3)
    output[:3, :3] = sample_img[seed_y:seed_y+3, seed_x:seed_x+3]
    
    # Gaussian kernel for weighting
    kernel = np.zeros((window_size, window_size))
    center = (window_size // 2, window_size // 2)
    for y in range(window_size):
        for x in range(window_size):
            kernel[y, x] = np.exp(-((y - center[0])**2 + (x - center[1])**2) / (2 * gaussian_sigma**2))
    kernel /= kernel.sum()
    
    # Priority queue for synthesis order (pixels adjacent to the seed)
    q = Queue()
    # Add pixels to the right and below the 3x3 seed
    for y in range(3):
        if 3 < w_out:
            q.put((y, 3))  # Right edge
    for x in range(3):
        if 3 < h_out:
            q.put((3, x))  # Bottom edge
    
    # Precompute all sample windows
    sample_windows = []
    for y in range(H - window_size + 1):
        for x in range(W - window_size + 1):
            sample_windows.append(sample_img[y:y+window_size, x:x+window_size])
    
    # Track synthesized pixels to avoid reprocessing
    synthesized_mask = np.zeros((h_out, w_out), dtype=bool)
    synthesized_mask[:3, :3] = True  # Mark seed as synthesized
    
    # Synthesize pixels
    with tqdm(total=h_out * w_out - 9) as pbar:
        while not q.empty():
            y, x = q.get()
            
            if synthesized_mask[y, x]:
                continue  # Skip already synthesized
            
            # Extract neighborhood (only known pixels)
            neighborhood = np.zeros((window_size, window_size))
            mask = np.zeros((window_size, window_size), dtype=bool)
            for dy in range(-window_size//2, window_size//2 + 1):
                for dx in range(-window_size//2, window_size//2 + 1):
                    ny = y + dy
                    nx = x + dx
                    if 0 <= ny < h_out and 0 <= nx < w_out and synthesized_mask[ny, nx]:
                        neighborhood[dy + window_size//2, dx + window_size//2] = output[ny, nx]
                        mask[dy + window_size//2, dx + window_size//2] = True
            
            # Skip if no known pixels
            if not mask.any():
                q.put((y, x))  # Requeue for later
                continue
            
            # Compute distances to sample windows
            min_dist = float('inf')
            best_matches = []
            for sw in sample_windows:
                # Compute Gaussian-weighted SSD over known pixels
                diff = (neighborhood[mask] - sw[mask]) ** 2
                dist = np.sum(diff * kernel[mask]) / np.sum(mask)
                if dist < min_dist:
                    min_dist = dist
                    best_matches = [sw]
                elif dist == min_dist:
                    best_matches.append(sw)
            
            # Find all matches within threshold
            valid_matches = [sw for sw in best_matches if min_dist <= threshold]
            if not valid_matches:
                valid_matches = best_matches  # Fallback to best match
            
            # Randomly select a match and assign its center
            selected = valid_matches[np.random.randint(len(valid_matches))]
            cy, cx = window_size//2, window_size//2
            output[y, x] = selected[cy, cx]
            synthesized_mask[y, x] = True
            pbar.update(1)
            
            # Add unsynthesized neighbors to the queue
            for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                ny, nx = y + dy, x + dx
                if 0 <= ny < h_out and 0 <= nx < w_out and not synthesized_mask[ny, nx]:
                    q.put((ny, nx))
    
    return output

  0%|          | 0/4087 [22:12<?, ?it/s]


ModuleNotFoundError: No module named 'scipy'

In [31]:
# Load sample texture
sample = cv2.imread('./images/1.png', cv2.IMREAD_GRAYSCALE)

# Synthesize a 128x128 texture with window_size=11
synthesized = efros_leung_synthesis(sample, (256, 256), window_size=11, threshold=0.1)

# Save result
cv2.imwrite('synthesized_texture.jpg', synthesized)

256 256 8




  0%|          | 0/65527 [00:00<?, ?it/s]

already synthesized 0 0
already synthesized 0 1
already synthesized 0 2
already synthesized 1 0
already synthesized 1 2
already synthesized 2 0
already synthesized 2 1
already synthesized 2 2


True